In [ ]:
#| default_exp helpers

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.
[INFO] numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] numexpr.utils: NumExpr defaulting to 8 threads.
[INFO] airt.keras.helpers: Using a single GPU #0 with memory_limit 1024 MB


In [ ]:
#| export

import os
import random
import string
import re
from contextlib import contextmanager
from os import environ
from pathlib import Path
from typing import *

import pandas as pd
import requests
from fastcore.utils import *
from passlib.context import CryptContext
from sqlmodel import Session

import airt_service.sanitizer
from airt.logger import get_logger

In [ ]:
import tempfile
from time import sleep

import pytest
import numpy as np
import dask.dataframe as dd
from sqlmodel import select
from _pytest.monkeypatch import MonkeyPatch

from airt.remote_path import RemotePath
from airt_service.data.datablob import FromLocalRequest, from_local_start_route
from airt_service.data.utils import create_db_uri_for_s3_datablob
from airt_service.db.models import (
    create_user_for_testing,
    get_session,
    get_session_with_context,
    User,
    DataBlob,
)

[INFO] airt.executor.subcommand: Module loaded.


In [ ]:
old_setattr = MonkeyPatch.setattr


@patch
def setattr(self: MonkeyPatch, *args, **kwargs):
    global logger
    old_setattr(self, *args, **kwargs)
    logger = get_logger(__name__)

In [ ]:
test_username = create_user_for_testing()
display(test_username)

'jiqkfavmfz'

In [ ]:
#| exporti

logger = get_logger(__name__)

In [ ]:
#| export

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

In [ ]:
#| export


def get_password_hash(password: str) -> str:
    """Get the hash for a given password

    Args:
        password: Password to be hashed as a string

    Returns:
        The hashed password as a string
    """
    return pwd_context.hash(password)


def verify_password(plain_password: str, hashed_password: str) -> bool:
    """Validate if the hashed password is derived from the plain password

    Args:
        plain_password: Plain password as a string
        hashed_password: Hashed password stored in the database as a string

    Returns:
        True, if the hashed password is derived from the plain password else False
    """
    return pwd_context.verify(plain_password, hashed_password)

In [ ]:
test_password = "Welcome123"

actual_password_hash = get_password_hash(test_password)
display(actual_password_hash)

assert verify_password(test_password, actual_password_hash)

'$2b$12$3K9c9V0LGe99CXn5h7rIDezQpb7Ur2wbR6jl4QzP5a9g2f3HaaQaW'

In [ ]:
#| export


def get_storage_path() -> Path:
    """Get the root storage_path to store datasource, models, predictions in local

    Returns:
        The root storage path
    """
    storage_path = Path(os.environ.get("STORAGE_PATH", "./storage"))
    storage_path = storage_path.absolute()
    storage_path.mkdir(parents=True, exist_ok=True)
    return storage_path

In [ ]:
actual = get_storage_path()
display(actual)
assert (actual == Path("/tf/airt-service/storage")) or (
    actual == Path("./storage").absolute()
)
assert actual.exists()

Path('/tf/airt-service/storage')

In [ ]:
#| export


def get_datasource_path() -> Path:
    """Get a local path to store the datasources

    Returns:
        A path to store the datasources
    """
    storage_path = get_storage_path()
    datasource_path = storage_path / "datasource"
    datasource_path.mkdir(parents=False, exist_ok=True)
    return datasource_path

In [ ]:
expected = get_storage_path() / "datasource"
actual = get_datasource_path()
display(actual)
assert actual == expected
assert actual.exists()

Path('/tf/airt-service/storage/datasource')

In [ ]:
#| export


def get_model_path() -> Path:
    """Get a local path to store the models

    Returns:
        A path to store the models
    """
    storage_path = get_storage_path()
    model_path = storage_path / "model"
    model_path.mkdir(parents=False, exist_ok=True)
    return model_path

In [ ]:
expected = get_storage_path() / "model"
actual = get_model_path()
display(actual)
assert actual == expected
assert actual.exists()

Path('/tf/airt-service/storage/model')

In [ ]:
#| export


def get_prediction_path() -> Path:
    """Get a local path to store the predictions

    Returns:
        A path to store the predictions
    """
    storage_path = get_storage_path()
    prediction_path = storage_path / "prediction"
    prediction_path.mkdir(parents=False, exist_ok=True)
    return prediction_path

In [ ]:
expected = get_storage_path() / "prediction"
actual = get_prediction_path()
display(actual)
assert actual == expected
assert actual.exists()

Path('/tf/airt-service/storage/prediction')

In [ ]:
#| export


def generate_random_string(length: int = 6) -> str:
    """Generate a random string of the given length

    Args:
        length: Length of the random string. If not set, then the default value 6 will be used.

    Returns:
        A random string of the given length
    """
    return "".join(
        random.choice(string.ascii_uppercase + string.digits)  # nosec B311
        for _ in range(length)
    )

In [ ]:
actual = generate_random_string(length=10)
display(actual)
assert len(actual) == 10

'B86264W1ER'

In [ ]:
#| export


@contextmanager
def set_env_variable_context(variable: str, value: str) -> Iterator[None]:
    old_value = environ[variable] if variable in environ else None
    environ[variable] = value
    yield
    if old_value is None:
        del environ[variable]
    else:
        environ[variable] = old_value

In [ ]:
# test JOB_EXECUTOR is not set already scenario
assert "SET_AND_TEST_ENV_VARIABLE" not in environ
with set_env_variable_context(variable="SET_AND_TEST_ENV_VARIABLE", value="something"):
    assert environ["SET_AND_TEST_ENV_VARIABLE"] == "something"
assert "SET_AND_TEST_ENV_VARIABLE" not in environ

# test JOB_EXECUTOR is set already using first with statement scenario
assert "SET_AND_TEST_ENV_VARIABLE" not in environ
with set_env_variable_context(variable="SET_AND_TEST_ENV_VARIABLE", value="something"):
    assert environ["SET_AND_TEST_ENV_VARIABLE"] == "something"

    with set_env_variable_context(
        variable="SET_AND_TEST_ENV_VARIABLE", value="different_value"
    ):
        assert environ["SET_AND_TEST_ENV_VARIABLE"] == "different_value"

    assert environ["SET_AND_TEST_ENV_VARIABLE"] == "something"
assert "SET_AND_TEST_ENV_VARIABLE" not in environ

In [ ]:
#| export


@contextmanager
def commit_or_rollback(session: Session) -> Iterator[None]:
    """A context manager to commit the changes to the database. In the case of an exception,
    the database will be rollback to the previous state.

    Args:
        session: Current session object
    """
    try:
        yield
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

In [ ]:
with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()

    uri = "s3://bucket"
    # test_tag = Tag.get_by_name(name="test", session=session)
    db_uri = create_db_uri_for_s3_datablob(
        uri=uri,
        access_key="access",
        secret_key="secret",
    )

    def test_commit_or_rollback(raise_exception: bool):
        with commit_or_rollback(session):
            datablob = DataBlob(
                type="s3",
                uri=db_uri,
                source=uri,
                cloud_provider="aws",
                region="eu-west-1",
                total_steps=1,
                user=user,
                #         tags=[test_tag],
            )
            session.add(datablob)
            assert datablob.id is None
            if raise_exception:
                raise ValueError("I had one job and I failed")

        return datablob

    # positive case
    datablob = test_commit_or_rollback(False)
    display(datablob)
    assert datablob.id

    # negative case
    with pytest.raises(ValueError):
        test_commit_or_rollback(True)

    display("ok")

DataBlob(id=26, uuid=UUID('768d6003-69be-406c-a55b-204782ed3d95'), type='s3', uri='s3://****************************************@bucket', source='s3://bucket', total_steps=1, completed_steps=0, folder_size=None, cloud_provider=<CloudProvider.aws: 'aws'>, region='eu-west-1', error=None, disabled=False, path=None, created=datetime.datetime(2022, 10, 20, 7, 59, 22), user_id=121, pulled_on=None, tags=[])

'ok'

In [ ]:
#| export


def truncate(s: str, length: int = 255) -> str:
    """Truncate the string to a given length

    Args:
        s: String to truncate
        length: Length to truncate the string

    Returns:
        The truncated string
    """
    return s[:length]

In [ ]:
test_cases = [
    {
        "s": "Error; something went wrong",
        "length": 255,
        "expected": "Error; something went wrong",
    },
    {
        "s": "Error; something went wrong",
        "length": 5,
        "expected": "Error",
    },
    {
        "s": "-" * 2000,
        "length": 255,
        "expected": "-" * 255,
    },
]

for case in test_cases:
    actual = truncate(s=case["s"], length=case["length"])
    display(f"{actual=}")
    assert actual == case["expected"]

"actual='Error; something went wrong'"

"actual='Error'"

"actual='---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------'"

In [ ]:
#| export


def df_to_dict(df: pd.DataFrame) -> Dict[str, Any]:
    """Convert pandas dataframe to dict

    Args:
        df: Input dataframe

    Returns:
        A dict the data and dtypes
    """
    d = {
        "data": df.to_dict("tight"),
        "dtypes": df.dtypes.apply(lambda x: str(x)).to_dict(),
    }
    return d


def dict_to_df(d: Dict[str, Any]) -> pd.DataFrame:
    """Convert the dict into a pandas dataframe

    Args:
        d: Dict containing the data and dtypes

    Returns:
        The pandas dataframe constructed from the dict
    """
    data = d["data"]
    dtypes = d["dtypes"]
    df = pd.DataFrame.from_dict(data, orient="tight")
    for k, v in dtypes.items():
        df[k] = df[k].astype(v)
    return df

In [ ]:
df = pd.util.testing.makeDataFrame().set_index("A")
for c in df.columns:
    df[f"{c}_float32"] = df[c].astype("float32")
    df[f"{c}_int32"] = df[c].astype("int32")
    df[f"{c}_bool"] = df[c].astype("bool")
df["ts"] = np.datetime64("now")

ddf = dd.from_pandas(df, npartitions=4)
ddf_head = ddf.head(10)
s = df_to_dict(ddf_head)
actual = dict_to_df(s)

pd.testing.assert_frame_equal(ddf_head, actual)
display(actual)

/root/.local/lib/python3.8/site-packages/pandas/util/__init__.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing
/root/.local/lib/python3.8/site-packages/dask/dataframe/core.py:7606: UserWarning: Insufficient elements for `head`. 10 elements requested, only 8 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,B,C,D,B_float32,B_int32,B_bool,C_float32,C_int32,C_bool,D_float32,D_int32,D_bool,ts
A,,,,,,,,,,,,,
-2.209765,1.980765,1.789313,0.844521,1.980765,1,True,1.789313,1,True,0.844521,0,True,2022-10-20 07:59:21
-1.584939,-0.367534,1.654168,0.939094,-0.367534,0,True,1.654168,1,True,0.939094,0,True,2022-10-20 07:59:21
-1.546101,-0.355850,-0.642616,1.283032,-0.355850,0,True,-0.642616,0,True,1.283032,1,True,2022-10-20 07:59:21
-1.135242,-0.579568,-1.526793,-0.832108,-0.579568,0,True,-1.526793,-1,True,-0.832108,0,True,2022-10-20 07:59:21
-1.089560,0.625968,-0.205677,-1.874327,0.625968,0,True,-0.205677,0,True,-1.874327,-1,True,2022-10-20 07:59:21
-0.927017,0.879269,1.158200,-0.720700,0.879269,0,True,1.158200,1,True,-0.720700,0,True,2022-10-20 07:59:21
-0.833045,0.434356,0.661008,-1.260591,0.434355,0,True,0.661008,0,True,-1.260591,-1,True,2022-10-20 07:59:21
-0.757721,-0.345270,0.451236,0.868114,-0.345270,0,True,0.451236,0,True,0.868114,0,True,2022-10-20 07:59:21


In [ ]:
#| export


def _detect_sql_code_injection(s: str) -> bool:
    """Check if the given string contains SQL code injection

    Args:
        s: String to validate

    Returns:
        True, if the given string contains SQL code injection
    """
    # https://larrysteinle.com/2011/02/20/use-regular-expressions-to-detect-sql-code-injection/
    regex_text = "('(''|[^'])*')|(;)|(\b(ALTER|CREATE|DELETE|DROP|EXEC(UTE){0,1}|INSERT( +INTO){0,1}|MERGE|SELECT|UPDATE|UNION( +ALL){0,1})\b)"
    return bool(re.search(regex_text, s))

In [ ]:
# unsafe inputs
unsafe_inputs = ["'; SELECT * FROM test ;--", "10; DROP TABLE test /*"]

# safe inputs
safe_inputs = [
    "index_col",
    "index-col",
    "indexCol",
    "revenue_10",
    "string with space",
    "str!@$",
    "$str",
    "%%test%%",
    "SELECT a FROM test",
]

for i in unsafe_inputs:
    assert _detect_sql_code_injection(i), i

for i in safe_inputs:
    assert not _detect_sql_code_injection(i), i

In [ ]:
#| export


def validate_user_inputs(xs: List[str]) -> None:
    """Validate the user input for SQL code injection

    Args:
        xs: List of strings

    Raises:
        ValueError: If the list contains SQL code injection
    """
    for i in xs:
        if _detect_sql_code_injection(i):
            raise ValueError(f"The input {i} is invalid. SQL code injection detected.")

In [ ]:
valid_inputs = ["index_col", "index-col", "indexCol", "revenue_10"]
validate_user_inputs(valid_inputs)

In [ ]:
invalid_inputs = ["index_col", "index-col", "indexCol", "'; SELECT * FROM test ;--"]

with pytest.raises(ValueError) as e:
    validate_user_inputs(invalid_inputs)

assert (
    "The input '; SELECT * FROM test ;-- is invalid. SQL code injection detected."
    == str(e.value)
)

In [ ]:
#| export


def get_attr_by_name(xs: Dict[str, Any], attr_name: str) -> Union[str, None]:
    """Get the attribute by name from the input dictionary

    The input dictionary may contain nested objects, and the attribute may present at the root
    level or nested within the dictionary. This function searches the entire dictionary and
    returns the attribute's value if a match is found; otherwise, it returns None.

    Args:
        xs: Input dictionary
        attr_name: The name of the attribute for which the value must be retrieved

    Returns:
        If the given attribute name is found in the input dictionary, the value of the attribute will be returned; otherwise, None
    """
    if attr_name in xs:
        ret_val = xs[attr_name]
    else:
        ret_val = next(
            (getattr(v, attr_name) for v in xs.values() if hasattr(v, attr_name)), None
        )
    return ret_val

In [ ]:
kwargs = {
    "otp": "123456",
    "user": "user",
    "session": "session",
    "random": "asd",
    "random_": "asd",
}
expected = "123456"
actual = get_attr_by_name(kwargs, "otp")
display(actual)
assert actual == expected


class Test:
    otp = "123456"
    username = "username"


kwargs = {
    "user": "user",
    "session": "session",
    "random": "asd",
    "random_": "asd",
    "d": Test,
}
expected = "123456"
actual = get_attr_by_name(kwargs, "otp")
display(actual)
assert actual == expected

expected = "username"
actual = get_attr_by_name(kwargs, "username")
display(actual)
assert actual == expected


kwargs = {
    "user": "user_value",
    "session": "session_value",
    "random": "asd",
    "random_": "asd",
}
expected = None
actual = get_attr_by_name(kwargs, "otp")
display(actual)
assert actual == expected

actual = get_attr_by_name(kwargs, "user")
display(actual)
assert actual == "user_value"

'123456'

'123456'

'username'

None

'user_value'